<a href="https://colab.research.google.com/github/Eylz-Liu/Mod-le-vol-locale/blob/main/notebook%20in%20R%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
rm(list=ls())
library(ggplot2)
#data=read.csv2("V:/01. DEAA/02.8 Annuel 311224/00. GSE/01. Calibrage GSE/01. Act/Calibrage de la fonction de vol/Model Vol act/Data_D.csv")

# Création du DataFrame
data <- data.frame(
  Jour = 1:10,
  Vol = c(13.633, 13.513, 13.785, 13.112, 12.657, 12.555, 13.089, 13.724, 12.974, 12.496)
)

# Affichage
print(data)


   Jour    Vol
1     1 13.633
2     2 13.513
3     3 13.785
4     4 13.112
5     5 12.657
6     6 12.555
7     7 13.089
8     8 13.724
9     9 12.974
10   10 12.496


In [5]:
k=0
beta=c(1)
V0=c(1)
Tau=c(1)
Vol_LT=mean(data$Vol)
Ent_Inf=Vol_LT-2
Ent_Sup=Vol_LT+2


In [ ]:
#Borne_inf=Vol_LT-0.1
#Borne_sup=Vol_LT+0.1
for ( k in 1 : (nrow(data)-1)){

  j=k+1
  if(data$Vol[k]<Ent_Inf ||data$Vol[k]>Ent_Sup)
  {
    Borne_inf=data$Vol[k]+(Vol_LT-data$Vol[k])*0.5-0.5
    Borne_sup=data$Vol[k]+(Vol_LT-data$Vol[k])*0.5+0.5

    while((data$Vol[j]<Borne_inf || data$Vol[j]>Borne_sup) & j<nrow(data))
    {
      j=j+1
    }
    if (j<nrow(data))
    {
      e=  abs(data$Vol[j] - Vol_LT)
      x=-log((abs(data$Vol[k]-Vol_LT)-e)/abs(data$Vol[k]-Vol_LT)) * (j-k)
      beta=c(beta,x)
      V0=c(V0,data$Vol[k])
      Tau=c(Tau,j-k)
    }
  }
}

beta=beta[c(-1)]
V0=V0[c(-1)]
Tau=Tau[c(-1)]
beta=beta/21
res=data.frame(cbind(V0,beta,Tau))
rm(plot)
plot=ggplot(res,aes(x=V0,y=beta))+geom_point(size=1,color="#CC6666")
plot=plot+geom_segment(aes(x=10,y=mean(beta),xend=60,yend=mean(beta)),linetype="dashed",size=1.5)


In [ ]:
#geom_hline(yintercept=mean(beta))
#plot=plot+geom_density_2d()
# Gradient de couleur
plot=plot+ylim(0,20)
#+ stat_density_2d(aes(fill = ..level..), geom="polygon")+ scale_fill_gradient(low="blue", high="red")+ylim(0,10)
plot
write.csv2(res,file="res_VF.csv")

######### IC de la moyenne ###############

NB=10000
dis_theta=vector(length = NB)
for (j in 1 : NB)
{
  boot_indice=sample(1:length(data$Vol),replace = T)
  dis_theta[j]=mean(data$Vol[boot_indice])
}
IC_sup=quantile(dis_theta,c(0.975))
IC_inf=quantile(dis_theta,c(0.025))
dis_data=data.frame(dis_theta)

rm(hist)
hist=ggplot(dis_data,aes(x=dis_theta))+  geom_histogram(aes(y=..density..), colour="black", fill="white")+
  geom_density(alpha=.2, fill="#FF6666")
hist=hist+geom_vline(aes(xintercept=mean(dis_theta)),
                     color="blue", linetype="dashed", size=1)
hist=hist+geom_vline(aes(xintercept=IC_sup),
                     color="red", linetype="dashed", size=0.5)
hist=hist+geom_vline(aes(xintercept=IC_inf),
                     color="red", linetype="dashed", size=0.5)
hist=hist+xlab("Moyenne du VCAC")
hist